In [8]:
from datasets import load_dataset
import pandas as pd


/Users/krithis/Documents/nlp_topic_modeling/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:

#https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023

data = load_dataset('McAuley-Lab/Amazon-Reviews-2023','raw_review_Gift_Cards',trust_remote_code=True)
#meta=load_dataset('McAuley-Lab/Amazon-Reviews-2023','raw_meta_Gift_Cards',trust_remote_code=True)

In [10]:
#sample what one row of the data looks like
print(data["full"][0])

{'rating': 5.0, 'title': 'Great gift', 'text': 'Having Amazon money is always good.', 'images': [], 'asin': 'B00IX1I3G6', 'parent_asin': 'B00IX1I3G6', 'user_id': 'AHZ6XMOLEWA67S3TX7IWEXXGWSOA', 'timestamp': 1549866158332, 'helpful_vote': 0, 'verified_purchase': True}


In [11]:
dataset=data["full"]
print(dataset)


Dataset({
    features: ['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase'],
    num_rows: 152410
})


In [12]:

# Create an empty list to store rows
rows = []
for review in dataset:
    rating=review['rating']
    title=review['title']
    text=review['text']
    images=review['images']
    asin=review['asin']
    parent_asin=review['parent_asin']
    user_id=review['user_id']
    timestamp=review['timestamp']
    helpful_vote=review['helpful_vote']
    verified_purchase=review['verified_purchase']
    rows.append([rating, title, text, images, asin, parent_asin, user_id, timestamp, helpful_vote, verified_purchase])

reviews = pd.DataFrame(rows, columns=['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase'])


In [13]:
reviews.head()


,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5.0,Great gift,Having Amazon money is always good.,[],B00IX1I3G6,B00IX1I3G6,AHZ6XMOLEWA67S3TX7IWEXXGWSOA,1549866158332,0,True
1,5.0,amazon gift card,Always the perfect gift. I have never given o...,[],B005ESMMWW,B005ESMMWW,AFZUK3MTBIBEDQOPAK3OATUOUKLA,1599875158120,0,False
2,5.0,perfect gift,When you have a person who is hard to shop for...,[],B01K8RIM5Y,B005S28ZES,AFZUK3MTBIBEDQOPAK3OATUOUKLA,1535939929239,27,True
3,5.0,Nice looking,The tin is a nice touch and pretty large. It'...,[],B0091JKVU0,B00ADR2LV6,AFZUK3MTBIBEDQOPAK3OATUOUKLA,1418439577000,0,False
4,1.0,Not $10 Gift Cards,I bought this pack of Starbucks Gift cards in ...,[],B00FTGTM5E,B00FTGTIOE,AH5L7ILVA6HYLZOUZIQAWNHVVK3A,1638068808115,2,True


In [14]:
#there is no 0.5 scale
reviews['rating'].value_counts()


rating
5.0    128248
1.0     12326
4.0      6692
3.0      3271
2.0      1873
Name: count, dtype: int64

In [15]:
#liklihood to recommend or NPS is produced on a scale of 1 - 10 , may not be relavent, NPS= (Promoters - Detractors)
#instead, we can take a look at, in eah star category what is the customer talking about?
#start with data preprocessing

#check how many product categories are there - 1137
reviews['parent_asin'].value_counts()


parent_asin
B00IX1I3G6    36863
B00ADR2LV6    14912
B077N4CNVJ     6940
B086KKT3RX     6635
B018F4M89S     6329
              ...  
B002YEWXZU        1
B00ONCTANE        1
B06XW61V3B        1
B002SC04U4        1
B00O988YK2        1
Name: count, Length: 1137, dtype: int64

In [16]:
#start with one category 
df_cat1=reviews[(reviews['parent_asin']=='B00IX1I3G6')]

#drop columns not required for this analysis
df_cat1.drop(['images'],inplace=True,axis=1)

/var/folders/8w/mv0nvzh126z_rwpbhvx_k4mh0000gn/T/ipykernel_55036/617545395.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat1.drop(['images'],inplace=True,axis=1)


In [17]:
df_cat1.head()
df_cat1['asin'].value_counts()

asin
B00IX1I3G6    36863
Name: count, dtype: int64

In [18]:
df_cat1['parent_asin'].value_counts()

parent_asin
B00IX1I3G6    36863
Name: count, dtype: int64

In [19]:
#asin=parent asin
df_cat1.drop(['asin'],inplace=True,axis=1)
df_cat1.head()

/var/folders/8w/mv0nvzh126z_rwpbhvx_k4mh0000gn/T/ipykernel_55036/3461983026.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat1.drop(['asin'],inplace=True,axis=1)


,rating,title,text,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5.0,Great gift,Having Amazon money is always good.,B00IX1I3G6,AHZ6XMOLEWA67S3TX7IWEXXGWSOA,1549866158332,0,True
6,5.0,Easy gift,Great and easy gift,B00IX1I3G6,AFSCQMP2EPYLGJN7OBTMEXCBE2OQ,1602104097716,0,True
14,5.0,Easy..,Reloading is easy. Shopping is easy. Amazon is...,B00IX1I3G6,AFO3G62P2JXCNMWZTAIB56KPG56A,1466368648000,0,True
31,5.0,So easy,Easy to do,B00IX1I3G6,AEINWALDRYT4NTF3M5NC2WZ5YQNA,1587064572520,0,True
32,2.0,Mistake,Hit button by mistake n deduction from credit ...,B00IX1I3G6,AHVPSZCUQH3UMQCGQSDZ7BZVUHNQ,1576194477554,0,True


In [20]:
df_cat1.shape #36.8K reviews 

#check if there are any reviews with empty text



(36863, 8)

In [21]:
df_cat1['text'].isna().any()


False

In [22]:
df_cat1['text'].isnull().any()

False

In [23]:
#get an understanding of some general themes in the text
#pre-proessing - text cleaning, tokenization,lowercasing,stopword removal  
#if words are very specific to your business, you can build a custom dictionary of stop words
import nltk
import ssl
#bash '/Applications/Python 3.12/Install Certificates.command'

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('all')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/krithis/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/krithis/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/krithis/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/krithis/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /Users/krithis/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading pa

True

In [24]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


In [25]:
# Preprocessing

# Define the path to your stop words file
stop_words_file = 'english_stopwords'
# Read the stop words from the file
with open(stop_words_file, 'r') as file:
    stop_words = file.readlines()
# Strip newline characters and any leading/trailing whitespace from each stop word
stop_words = [word.strip() for word in stop_words]

#list of stop_words
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [26]:
lemmatizer = WordNetLemmatizer()


In [27]:

def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

df_cat1['processed_text'] = df_cat1['text'].apply(preprocess_text)

# nltk.download(r'lib/python3.12/site-packages/nltk/tokenize/punkt.py')
# nltk.download('wordnet')

/var/folders/8w/mv0nvzh126z_rwpbhvx_k4mh0000gn/T/ipykernel_55036/1856114349.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat1['processed_text'] = df_cat1['text'].apply(preprocess_text)


In [28]:
df_cat1.head()

,rating,title,text,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,processed_text
0,5.0,Great gift,Having Amazon money is always good.,B00IX1I3G6,AHZ6XMOLEWA67S3TX7IWEXXGWSOA,1549866158332,0,True,"[amazon, money, always, good]"
6,5.0,Easy gift,Great and easy gift,B00IX1I3G6,AFSCQMP2EPYLGJN7OBTMEXCBE2OQ,1602104097716,0,True,"[great, easy, gift]"
14,5.0,Easy..,Reloading is easy. Shopping is easy. Amazon is...,B00IX1I3G6,AFO3G62P2JXCNMWZTAIB56KPG56A,1466368648000,0,True,"[reloading, easy, shopping, easy, amazon, easy]"
31,5.0,So easy,Easy to do,B00IX1I3G6,AEINWALDRYT4NTF3M5NC2WZ5YQNA,1587064572520,0,True,[easy]
32,2.0,Mistake,Hit button by mistake n deduction from credit ...,B00IX1I3G6,AHVPSZCUQH3UMQCGQSDZ7BZVUHNQ,1576194477554,0,True,"[hit, button, mistake, n, deduction, credit, c..."


In [32]:

from sklearn.feature_extraction.text import CountVectorizer
# Create Document-Term Matrix
vectorizer = CountVectorizer(tokenizer=lambda x: x, lowercase=False)
dtm = vectorizer.fit_transform(df_cat1['processed_text'])



/Users/krithis/Documents/nlp_topic_modeling/.venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [39]:

from gensim import corpora, models
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel

#The scipy.linalg functions tri, triu & tril are deprecated and will be removed in SciPy 1.13. pip install scipy==1.13.0



ImportError: cannot import name 'triu' from 'scipy.linalg' (/Users/krithis/Documents/nlp_topic_modeling/.venv/lib/python3.12/site-packages/scipy/linalg/__init__.py)

In [ ]:

# Topic Modeling
num_topics = 10
corpus = corpora.MmCorpus(dtm)
id2word = dict((v, k) for k, v in vectorizer.vocabulary_.items())

lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics)

# Evaluate the Model (Optional)
coherence_model_lda = CoherenceModel(model=lda_model, texts=df['processed_text'], dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print(f'Coherence Score: {coherence_lda}')

# Interpret the Results
for topic_id, topic_words in lda_model.print_topics():
    print(f'Topic {topic_id}: {topic_words}')